#  Manipulating DataFrames with Pandas

### *Copyright 2021-2022 Dr. George Papagiannakis,  papagian@csd.uoc.gr*
*All Rights Reserved*
### *University of Crete & Foundation for Research & Technology - Hellas (FORTH)*

This notebook is also based on parts of [Pandas tutorial](http://gregreda.com/2013/10/26/intro-to-pandas-data-structures/) by [Greg Reda](http://www.gregreda.com). 

---

In [1]:
import pandas as pd
import numpy as np

## Joining

Throughout an analysis, we'll often need to merge/join datasets as data is typically stored in a [relational](http://en.wikipedia.org/wiki/Relational_database) manner.

Our MovieLens data is a good example of this - a rating requires both a user and a movie, and the datasets are linked together by a key — in this case, the *user_id* and *movie_id*. It's possible for a user to be associated with zero or many ratings and movies. Likewise, a movie can be rated zero or many times, by a number of different users.

Like SQL's JOIN clause, `pandas.merge` allows two DataFrames to be joined on one or more keys. The function provides a series of parameters `(on, left_on, right_on, left_index, right_index)` allowing you to specify the columns or indexes on which to join.

By default, `pandas.merge` operates as an *inner join*, which can be changed using the `how` parameter.

From the function's docstring:

> how : {'left', 'right', 'outer', 'inner'}, default 'inner'
>    * left: use only keys from left frame (SQL: left outer join)
>    * right: use only keys from right frame (SQL: right outer join)
>    * outer: use union of keys from both frames (SQL: full outer join)
>    * inner: use intersection of keys from both frames (SQL: inner join)

Below are some examples of what each look like.

In [2]:
left_frame = pd.DataFrame({'key': range(5), 
                           'left_value': ['a', 'b', 'c', 'd', 'e']})
right_frame = pd.DataFrame({'key': range(2, 7), 
                           'right_value': ['f', 'g', 'h', 'i', 'j']})
print(left_frame)
print('\n')
print(right_frame)

   key left_value
0    0          a
1    1          b
2    2          c
3    3          d
4    4          e


   key right_value
0    2           f
1    3           g
2    4           h
3    5           i
4    6           j


**inner join (default)**

In [3]:
pd.merge(left_frame, right_frame, on='key', how='inner')

,key,left_value,right_value
0,2,c,f
1,3,d,g
2,4,e,h


We lose values from both frames since certain keys do not match up.  The SQL equivalent is:

```
    SELECT left_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    INNER JOIN right_frame
        ON left_frame.key = right_frame.key;
```

Had our *key* columns not been named the same, we could have used the `left_on` and `right_on` parameters to specify which fields to join from each frame.
```python
    pd.merge(left_frame, right_frame, left_on='left_key', right_on='right_key')
```
Alternatively, if our keys were indexes, we could use the `left_index` or `right_index` parameters, which accept a True/False value. You can mix and match columns and indexes like so:
```python
    pd.merge(left_frame, right_frame, left_on='key', right_index=True)
```

**left outer join**

In [4]:
pd.merge(left_frame, right_frame, on='key', how='left')

,key,left_value,right_value
0,0,a,NaN
1,1,b,NaN
2,2,c,f
3,3,d,g
4,4,e,h


We keep everything from the left frame, pulling in the value from the right frame where the keys match up. The `right_value` is NULL where keys do not match (NaN).

SQL Equivalent:

    SELECT left_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    LEFT JOIN right_frame
        ON left_frame.key = right_frame.key;

**right outer join**

In [5]:
pd.merge(left_frame, right_frame, on='key', how='right')

,key,left_value,right_value
0,2,c,f
1,3,d,g
2,4,e,h
3,5,NaN,i
4,6,NaN,j


This time we've kept everything from the right frame with the `left_value` being NULL where the right frame's key did not find a match.

SQL Equivalent:

    SELECT right_frame.key, left_frame.left_value, right_frame.right_value
    FROM left_frame
    RIGHT JOIN right_frame
        ON left_frame.key = right_frame.key;

**full outer join**

In [6]:
pd.merge(left_frame, right_frame, on='key', how='outer')

,key,left_value,right_value
0,0,a,NaN
1,1,b,NaN
2,2,c,f
3,3,d,g
4,4,e,h
5,5,NaN,i
6,6,NaN,j


We've kept everything from both frames, regardless of whether or not there was a match on both sides. Where there was not a match, the values corresponding to that key are NULL.

SQL Equivalent (though some databases don't allow FULL JOINs (e.g. MySQL)):

    SELECT IFNULL(left_frame.key, right_frame.key) key
            , left_frame.left_value, right_frame.right_value
    FROM left_frame
    FULL OUTER JOIN right_frame
        ON left_frame.key = right_frame.key;

## Combining

pandas also provides a way to combine DataFrames along an axis — `pandas.concat`. While the function is equivalent to SQL's UNION clause, there's a lot more that can be done with it.

`pandas.concat` takes a list of Series or DataFrames and returns a Series or DataFrame of the concatenated objects. Note that because the function takes list, you can combine many objects at once.

In [7]:
pd.concat([left_frame, right_frame], sort=False)

,key,left_value,right_value
0,0,a,NaN
1,1,b,NaN
2,2,c,NaN
3,3,d,NaN
4,4,e,NaN
0,2,NaN,f
1,3,NaN,g
2,4,NaN,h
3,5,NaN,i
4,6,NaN,j


By default, the function will vertically append the objects to one another, combining columns with the same name. We can see above that values not matching up will be NULL.

Additionally, objects can be concatentated side-by-side using the function's `axis` parameter.

In [8]:
pd.concat([left_frame, right_frame], axis=1)

,key,left_value,key,right_value
0,0,a,2,f
1,1,b,3,g
2,2,c,4,h
3,3,d,5,i
4,4,e,6,j


`pandas.concat` can be used in a variety of ways; however, I've typically only used it to combine Series/DataFrames into one unified object. The [documentation](http://pandas.pydata.org/pandas-docs/stable/merging.html#concatenating-objects) has some examples on the ways it can be used.

## Grouping

Grouping in pandas took some time for me to grasp, but it's pretty awesome once it clicks.

pandas `groupby` method draws largely from the [split-apply-combine strategy for data analysis](http://www.jstatsoft.org/v40/i01/paper).  If you're not familiar with this methodology, I highly suggest you read up on it.  It does a great job of illustrating how to properly think through a data problem, which I feel is more important than any technical skill a data analyst/scientist can possess.

When approaching a data analysis problem, you'll often break it apart into manageable pieces, perform some operations on each of the pieces, and then put everything back together again (this is the gist split-apply-combine strategy). pandas `groupby` is great for these problems (R users should check out the [plyr](http://plyr.had.co.nz/) and [dplyr](https://github.com/hadley/dplyr) packages).

If you've ever used SQL's GROUP BY or an Excel Pivot Table, you've thought with this mindset, probably without realizing it.

Assume we have a DataFrame and want to get the average for each group - visually, the split-apply-combine method looks like this:

<img src="resources/pandas_grouping.png" width="320px">

The City of Chicago is kind enough to publish all city employee salaries to its open data portal. Let's go through some basic `groupby` examples using this data.

In [9]:
with open('data/city-of-chicago-salaries.csv') as f:
    print("\n".join(f.read().splitlines()[:3]))

Name,Position Title,Department,Employee Annual Salary
"AARON,  ELVIA J",WATER RATE TAKER,WATER MGMNT,$85512.00
"AARON,  JEFFERY M",POLICE OFFICER,POLICE,$75372.00


Since the data contains a dollar sign for each salary, python will treat the field as a series of strings. We can use the `converters` parameter to change this when reading in the file.

>converters : dict. optional
>* Dict of functions for converting values in certain columns. Keys can either be integers or column labels

In [10]:
headers = ['name', 'title', 'department', 'salary']
chicago = pd.read_csv('data/city-of-chicago-salaries.csv', 
                      header=0,
                      names=headers,
                      converters={'salary': lambda x: float(x.replace('$', ''))})
chicago.head()

,name,title,department,salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,85512.0
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,75372.0
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,80916.0
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,99648.0
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,89440.0


pandas `groupby` returns a DataFrameGroupBy object which has a variety of methods, many of which are similar to standard SQL aggregate functions.

In [11]:
by_dept = chicago.groupby('department')
by_dept

Calling `count` returns the total number of NOT NULL values within each column. If we were interested in the total number of records in each group, we could use `size`.

In [12]:
print(by_dept.count().head()) # NOT NULL records within each column
print('\n')
print(by_dept.size().tail()) # total records for each department

                   name  title  salary
department                            
ADMIN HEARNG         42     42      42
ANIMAL CONTRL        61     61      61
AVIATION           1218   1218    1218
BOARD OF ELECTION   110    110     110
BOARD OF ETHICS       9      9       9


department
PUBLIC LIBRARY     926
STREETS & SAN     2070
TRANSPORTN        1168
TREASURER           25
WATER MGMNT       1857
dtype: int64


Summation can be done via `sum`, averaging by `mean`, etc. (if it's a SQL function, chances are it exists in pandas). Oh, and there's median too, something not available in most databases.

In [13]:
print(by_dept.sum()[20:25]) # total salaries of each department
print('\n')
print(by_dept.mean()[20:25]) # average salary of each department
print('\n')
print(by_dept.median()[20:25]) # take that, RDBMS!

                       salary
department                   
HUMAN RESOURCES     4850928.0
INSPECTOR GEN       4035150.0
IPRA                7006128.0
LAW                31883920.2
LICENSE APPL COMM     65436.0


                         salary
department                     
HUMAN RESOURCES    71337.176471
INSPECTOR GEN      80703.000000
IPRA               82425.035294
LAW                70853.156000
LICENSE APPL COMM  65436.000000


                    salary
department                
HUMAN RESOURCES    68496.0
INSPECTOR GEN      76116.0
IPRA               82524.0
LAW                66492.0
LICENSE APPL COMM  65436.0


Operations can also be done on an individual Series within a grouped object. Say we were curious about the five departments with the most distinct titles — the pandas equivalent to:

    SELECT department, COUNT(DISTINCT title)
    FROM chicago
    GROUP BY department
    ORDER BY 2 DESC
    LIMIT 5;

pandas is a lot less verbose here:

In [14]:
by_dept.title.nunique().sort_values(ascending=False)[:5]

department
WATER MGMNT    153
TRANSPORTN     150
POLICE         130
AVIATION       125
HEALTH         118
Name: title, dtype: int64

## split-apply-combine

The real power of `groupby` comes from it's split-apply-combine ability.

What if we wanted to see the highest paid employee within each department. Given our current dataset, we'd have to do something like this in SQL:

    SELECT *
    FROM chicago c
    INNER JOIN (
        SELECT department, max(salary) max_salary
        FROM chicago
        GROUP BY department
    ) m
    ON c.department = m.department
    AND c.salary = m.max_salary;
    
This would give you the highest paid person in each department, but it would return multiple if there were many equally high paid people within a department.

Alternatively, you could alter the table, add a column, and then write an update statement to populate that column. However, that's not always an option.

_Note: This would be a lot easier in PostgreSQL, T-SQL, and possibly Oracle due to the existence of partition/window/analytic functions. I've chosen to use MySQL syntax throughout this tutorial because of it's popularity. Unfortunately, MySQL doesn't have similar functions._

Using `groupby` we can define a function (which we'll call `ranker`) that will label each record from 1 to N, where N is the number of employees within the department. We can then call `apply` to, well, _apply_ that function to each group (in this case, each department).

In [15]:
def ranker(df):
    """Assigns a rank to each employee based on salary, with 1 being the highest paid.
    Assumes the data is DESC sorted."""
    df['dept_rank'] = np.arange(len(df)) + 1
    return df

In [16]:
chicago.sort_values('salary', ascending=False, inplace=True)
chicago = chicago.groupby('department').apply(ranker)
print(chicago[chicago.dept_rank == 1].head(7))

                         name                     title      department  \
18039     MC CARTHY,  GARRY F  SUPERINTENDENT OF POLICE          POLICE   
8004           EMANUEL,  RAHM                     MAYOR  MAYOR'S OFFICE   
25588       SANTIAGO,  JOSE A         FIRE COMMISSIONER            FIRE   
763    ANDOLINO,  ROSEMARIE S  COMMISSIONER OF AVIATION        AVIATION   
4697     CHOUCAIR,  BECHARA N    COMMISSIONER OF HEALTH          HEALTH   
21971      PATTON,  STEPHEN R       CORPORATION COUNSEL             LAW   
12635      HOLT,  ALEXANDRA D                BUDGET DIR   BUDGET & MGMT   

         salary  dept_rank  
18039  260004.0          1  
8004   216210.0          1  
25588  202728.0          1  
763    186576.0          1  
4697   177156.0          1  
21971  173664.0          1  
12635  169992.0          1  


In [17]:
chicago[chicago.department == "LAW"][:5]

,name,title,department,salary,dept_rank
21971,"PATTON, STEPHEN R",CORPORATION COUNSEL,LAW,173664.0,1
6311,"DARLING, LESLIE M",FIRST ASST CORPORATION COUNSEL,LAW,149160.0,2
17680,"MARTINICO, JOSEPH P",CHIEF LABOR NEGOTIATOR,LAW,144036.0,3
22357,"PETERS, LYNDA A",CITY PROSECUTOR,LAW,139932.0,4
31383,"WONG JR, EDWARD J",DEPUTY CORPORATION COUNSEL,LAW,137076.0,5


We can now see where each employee ranks within their department based on salary.

## Further reading

Using pandas you can do any data manipulation you can do in excel or a database plus a whole lot more. The only limitation is that all the work must be able to be done in-memory on your computer.

You may have noticed the general process is to perform some manipulation and then call `df.head()` to see if you got what you want. In a real application you'll likely also be googling how to do certain operations or debug your error message, similar to a lot of other coding and analytical work.

* Check out more introductory notebooks in **Juno**!
* [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) 
* [Introduction to pandas by Chris Fonnesbeck](http://nbviewer.ipython.org/urls/gist.github.com/fonnesbeck/5850375/raw/c18cfcd9580d382cb6d14e4708aab33a0916ff3e/1.+Introduction+to+Pandas.ipynb) 
* [Pandas videos from PyCon](http://pyvideo.org/search?models=videos.video&q=pandas)
* [Pandas and Python top 10](http://manishamde.github.io/blog/2013/03/07/pandas-and-python-top-10/)
* [Practical pandas by Tom Augspurger (one of the pandas developers)](http://tomaugspurger.github.io/categories/pandas.html)

## Versions

In [18]:
%reload_ext version_information

%version_information pandas, numpy, xlrd

Software versions
Python 3.6.6+ 64bit [GCC 4.2.1 Compatible Apple LLVM 10.0.1 (clang-1001.0.46.4)]
IPython 7.3.0
OS Darwin 18.7.0 x86_64 64bit
pandas 0.25.0
numpy 1.16.4
xlrd 1.2.0
Wed Sep 04 21:58:12 2019 BST